# Mine Recognition

## Step 1: Import Libraries

In [1]:
import os
import json
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Step 2: Load and Prepare the Data

In [2]:
# Load the JSON file
with open('images/data.json', 'r') as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(data)
df['image_path'] = df['file_name']  # Ensure file names match your actual path

# Split data into train and validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

## Step 3: Create a Custom Dataset

In [3]:
class MineDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.dataframe.iloc[idx]['image_path'])
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.iloc[idx]['label']
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Initialize datasets and loaders
train_dataset = MineDataset(train_df, img_dir='images', transform=transform)
val_dataset = MineDataset(val_df, img_dir='images', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

## Step 4: Define the Model
Load a pre-trained model, such as ResNet18, and modify it for binary classification.

In [4]:
import torch.nn as nn

class MineClassifier(nn.Module):
    def __init__(self):
        super(MineClassifier, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 256)  # Adjust dimensions based on input size
        self.fc2 = nn.Linear(256, 1)
        
        # Activation function and dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Convolutional layers with pooling and ReLU activation
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        
        # Flatten the tensor for the fully connected layer
        x = x.view(-1, 128 * 28 * 28)  # Adjust this based on input dimensions
        
        # Fully connected layers with dropout
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

## Step 5: Set Loss Function and Optimizer
Binary cross-entropy loss with logits and Adam optimizer.

In [5]:
# Initialize the custom model
model = MineClassifier()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device

device(type='cuda')

## Step 6: Training Loop
Train the model and evaluate on the validation set after each epoch.

In [6]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    # Training phase
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float()
        
        optimizer.zero_grad()
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images).squeeze(1)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            
            # Store predictions and true labels
            preds = torch.sigmoid(outputs) > 0.5  # Apply threshold to get binary predictions
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader.dataset):.4f}, "
          f"Val Loss: {val_loss/len(val_loader.dataset):.4f}, "
          f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Epoch [1/20], Loss: 0.6928, Val Loss: 0.0500, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000


c:\Users\miche\anaconda3\envs\CV\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [2/20], Loss: 0.7628, Val Loss: 6.2032, Accuracy: 0.0000, Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch [3/20], Loss: 2.1849, Val Loss: 0.4469, Accuracy: 0.7500, Precision: 1.0000, Recall: 0.7500, F1 Score: 0.8571
Epoch [4/20], Loss: 0.4523, Val Loss: 0.0719, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000
Epoch [5/20], Loss: 0.4164, Val Loss: 0.0870, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000
Epoch [6/20], Loss: 0.2266, Val Loss: 0.1943, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000
Epoch [7/20], Loss: 0.0830, Val Loss: 0.2889, Accuracy: 0.7500, Precision: 1.0000, Recall: 0.7500, F1 Score: 0.8571
Epoch [8/20], Loss: 0.0568, Val Loss: 0.1578, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000
Epoch [9/20], Loss: 0.0477, Val Loss: 0.0525, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000
Epoch [10/20], Loss: 0.0051, Val Loss: 0.0187, Accuracy: 1.0000, Precisi

## Step 7: Predict on New Images
Now, we can classify new images that don't have labels.

In [10]:
def classify_image(model, image_path):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device
    
    with torch.no_grad():
        output = model(image)
        prediction = torch.sigmoid(output).item()  # Sigmoid to get probability
        return 1 if prediction > 0.5 else 0  # Threshold of 0.5 for binary classification

# Example of classifying a new image
new_image_path = 'images/unknown0.jpg'
prediction = classify_image(model, new_image_path)
print(f"Prediction: {'Mine' if prediction == 1 else 'No Mine'}")


Prediction: No Mine
